# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). 

Нужно построить модель для задачи классификации, которая выберет подходящий тариф. 

Предобработка данных не понадобится — мы её уже сделали.

Здесь нужно построить модель с максимально большим значением *accuracy*. 

Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Также нужно проверить *accuracy* на тестовой выборке.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Откройте-и-изучите-файл" data-toc-modified-id="Откройте-и-изучите-файл-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Откройте и изучите файл</a></span></li><li><span><a href="#Разбейте-данные-на-выборки" data-toc-modified-id="Разбейте-данные-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разбейте данные на выборки</a></span></li><li><span><a href="#Исследуйте-модели" data-toc-modified-id="Исследуйте-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследуйте модели</a></span></li><li><span><a href="#Проверьте-модель-на-тестовой-выборке" data-toc-modified-id="Проверьте-модель-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверьте модель на тестовой выборке</a></span></li><li><span><a href="#(бонус)-Проверьте-модели-на-адекватность" data-toc-modified-id="(бонус)-Проверьте-модели-на-адекватность-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>(бонус) Проверьте модели на адекватность</a></span></li></ul></div>

# Подключение библиотек

In [1]:
import pandas as pd

# Для деления на две выборки (Валидационную выборку)
from sklearn.model_selection import train_test_split
# Модель классификация
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

# Проверка на адекватность
from sklearn.dummy import DummyClassifier

# Функция для оценки правильности
from sklearn.metrics import accuracy_score

## Откройте и изучите файл

In [2]:
# Получаем датасет
try:
    df = pd.read_csv('/datasets/users_behavior.csv')
except:
    df = pd.read_csv('datasets/users_behavior.csv')

# Смотрим и изучаем датасет
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
# Проверяем пропущенные значения
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

Нет пропущенных значений, теперь переходим к следующему шагу

## Разбейте данные на выборки

In [5]:
# Устанавливаем признаки и цели обучения
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

Нам нужно отделить по 20% данных для валидационной и тестовой выборки, а 60% - обучающей выборки

In [6]:
# Разделяем исходные данные на обучающую, валидационную и тестовую выборки

# Как обычно, будем разделять 60% данных для обучающей выборки, а features_temp и target_temp будут временным датафреймом
features_train, features_temp, target_train, target_temp = train_test_split(features, target, test_size= (1 - 0.6), random_state=12345) 

# А тестовая и валидационная - по 20% данных
features_valid, features_test, target_valid, target_test = train_test_split(features_temp, target_temp, test_size=(1 - 0.5), random_state=12345) 

## Исследуйте модели

Нам требуется исследовать какими тарифом пользователь пользуются, а эти признаки являются категориальными, т.е. будем применять методом обучения для классификации

Исследуем качество модели разными способами

In [7]:
# Методом случайного леса

best_model = None
best_result = 0
for est in range(1, 11):
    # Обучим модель с заданным количеством деревьев
    model = RandomForestClassifier(random_state=12345, n_estimators=est) 
    
    # Обучим модель на тренировочной выборке
    model.fit(features_train, target_train) 
    
    # Посчитаем качество модели на валидационной выборке
    result = model.score(features_valid, target_valid) 
    
    if result > best_result:
        # Сохраним наилучшую модель
        best_model = model
        # Сохраним наилучшее значение метрики accuracy на валидационных данных
        best_result = result

In [8]:
print("Accuracy наилучшей модели на валидационной выборке:", best_result)

Accuracy наилучшей модели на валидационной выборке: 0.7853810264385692


In [9]:
# Методом решающего дерева

best_model = None
best_result = 0

for depth in range(1, 6):
    # Обучим модель с заданной глубиной дерева
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    
    # Обучим модель
    model.fit(features_train, target_train)
    
    # Получим предсказания модели
    predictions_valid = model.predict(features_valid)
    
    # Посчитаем качество модели
    result = accuracy_score(target_valid, predictions_valid) 
    
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
    
    print("max_depth =", depth, ": ", end='')
    print(result)

max_depth = 1 : 0.7542768273716952
max_depth = 2 : 0.7822706065318819
max_depth = 3 : 0.7853810264385692
max_depth = 4 : 0.7791601866251944
max_depth = 5 : 0.7791601866251944


In [10]:
print("Accuracy лучшей модели:", best_result, '\nГлубина обучения:', best_depth)

Accuracy лучшей модели: 0.7853810264385692 
Глубина обучения: 3


Здесь видно, что максимальная глубина 3 является оптимальной по сравнению остальных 

## Проверьте модель на тестовой выборке

In [11]:
# Обучим модель с заданным количеством деревьев
model = RandomForestClassifier(random_state=12345, n_estimators=est) 

# Обучим модель на тренировочной выборке
model.fit(features_train, target_train) 

# Получим предсказания обучающей модели
train_predictions = model.predict(features_train)

# Получим предсказания тестовой модели
test_predictions = model.predict(features_test)

In [12]:
print("Accuracy")
print("Обучающая выборка:", accuracy_score(target_train, train_predictions))
print("Тестовая выборка:", accuracy_score(target_test, test_predictions))

Accuracy
Обучающая выборка: 0.9823651452282157
Тестовая выборка: 0.7807153965785381


Здесь получилось, что обученная модель показалась себя намного лучше, ближе к 1.0, а тестовая справилась, но не намного хуже валидационного выборки, разница лишь в сотых, отн. погрешность около 0,6%

## (бонус) Проверьте модели на адекватность

In [13]:
# Устанавливаем стратегию прогноза как "самый частый случай", 
# т.е. частую метку класса в наблюдаемом аргументе target, 
# переданном для соответствия
dummy_clf = DummyClassifier(strategy = "most_frequent")

# Обучим модель
dummy_clf.fit(features_train, target_train)

# Создаем модель, указав стратению как "самый частый случай"
DummyClassifier(strategy = 'most_frequent')

# Получим предсказания модели
dummy_clf.predict(features_train)

# Посчитаем качество модели
dummy_clf.score(features_train, target_train)

0.6924273858921162

Получилось, что обученная модель не является идеальным по сравнению остальных

In [14]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64